# FIT5196 Assessment 1 Task 1


## Student Name: Ding Ding


## Student ID: 32049145


#### Environment: Python 3.9 and Anaconda 4.11.0


#### Libraries used:
#### re (for regular expression, included in Anaconda Python 3.9)

# 1. Introduction

This assignment aims to convert the semi-structured text data to an xml data format.There are 40 text files about 37.3MB in total, which should be transformed.


1. Merge all text files to one variable called input
     

# 2. Import library

In [1]:
import re

# 3. Loading data

In [2]:
#define a new variable called input
input = ''

#read the whole file and load into input
for i in range(1,41):
    with open(r'C:/Users/ROG/5196/32049145/32049145_task1_input' + str(i) + '.txt', 'r', encoding = 'utf-8') as fp:
        for line in fp:
            input += line
        fp.close()

# 4.Unified text format and avoid illegal characters for xml format

1. I grabbed document and used $ as the identifier first, and then I found that there are only a few similar variants of one feature.
2. I also found some problems on xml format that some symbols is illgeal, I also changed them to the correct format.

In [3]:
#keep the input unchanged
input1 = input

#use re function to delete several continous newline to one
output = re.sub(r'(?:\n)+', r' ', input1)

#unified all characters
output = re.sub(r'\$(?:rev[\w\.]+ID.): ([A-Z0-9]+) \$', r'$reviewer.ID: \1\n$', output)

output = re.sub(r'\$(?:p.*?ID.): ([A-Z0-9]+) \$', r'$product.ID: \1$', output)

output = re.sub(r'\$review(?:_date|Date).: (\d{2} \d{1,2}, \d{4}) \$', r'$reviewDate: \1$', output)

output = re.sub(r'(?:\$No. helps|\$Helpful\?).: (\[\d+, \d+]) \$', r'$helpful.no: \1$', output)

output = re.sub(r'(?:\$review_text.|\$REVIEW.): (.*?) \$', r'$review_text: \1$', output)

output = re.sub(r'(?:\$review_summary.|\$SUMMARY.): (.*?) \$r', r'$review_summary: \1\n$r', output)

output = re.sub(r'(?:\$reviewer.NAME|\$reviewerName|\$rev_NAME).: (.*?) \$', r'$', output)

#change all illegal characters to legal one
output = output.replace('&', ' &amp; ')
output = output.replace("'", " &apos; ")
output = output.replace('"', " &quot; ")
output = output.replace('>', ' &gt; ')
output = output.replace('<', ' &lt; ')

# 5. Change the text from string to a list and then to a dictionary
1. Split the string into a list with the identifier \n, so that every character can be a unique element
2. Find all reviewer.ID to put it as the key of the dictionary and others as the value
3. Extract the value of dictionary to several lists with the character as the identifier
4. Change the order of value in the dictonary

In [4]:
list_a = []
#Split the string into a list with the identifier \n
list_a = output.split('\n')

dict_output = dict()

#put reviewer.ID as key and others as value and if the reviewer.ID exists, add the other characters into the value
for i in range(len(list_a)):
    if '$reviewer.ID:' in list_a[i]:
        if list_a[i] not in dict_output.keys():
            dict_output[list_a[i]] = list_a[i+1]
        else:
            dict_output[list_a[i]] += list_a[i+1]

#split the value of the dictionary
for k,v in dict_output.items():
    dict_output[k] = re.split(r'(\$product.ID: |\$reviewDate: |\$helpful.no: |\$review_summary: |\$review_text: )', v)
    #remove the '' in the front
    while '' in dict_output[k]:
        dict_output[k].remove('')
    #add all later element in the list to the former
    for i in range(len(dict_output[k])):
        if i < len(dict_output[k]) - 1:
            dict_output[k][i] += dict_output[k][i+1]
    
    #delete all odd element of the list in the value of dictionary, which end with character
    dict_output_list = []
    for j in range(len(dict_output[k])):
        if j % 2 == 0:
            dict_output_list.append(dict_output[k][j])
    dict_output[k] = dict_output_list
    
    #define all variables needed later, pi represents productID and so on, and value ordered is the order in the correct way
    value_ordered = ''
    pi = []
    rd = []
    hn = []
    rt = []
    rs = []
    #if product.ID in the element of the list, change it to the xml format and so on
    for i in range(len(dict_output[k])):
        if "$product.ID:" in dict_output[k][i]:
            temp = ''
            temp = re.sub(r'\$product.ID: (.*?)', r'<reviews><review><productID>\1', dict_output[k][i])
            temp += '</productID>'
            pi.append(temp)
        elif "$reviewDate:" in dict_output[k][i]:
            temp = ''
            temp = re.sub(r'\$reviewDate: (.*?)', r'<review_date>\1', dict_output[k][i])
            temp += '</review_date>'
            rd.append(temp)
        elif "$helpful.no:" in dict_output[k][i]:
            temp = ''
            temp = re.sub(r'\$helpful.no: (.*?)', r'<review_helpful>\1', dict_output[k][i])
            temp += '</review_helpful>'
            hn.append(temp)
        elif "$review_text:" in dict_output[k][i]:
            temp = ''
            temp = re.sub(r'\$review_text: (.*?)', r'<review_text>\1', dict_output[k][i])
            temp += '</review_text>'
            rt.append(temp)
        elif "$review_summary:" in dict_output[k][i]:
            temp = ''
            temp = re.sub(r'\$review_summary: (.*?)', r'<review_summary>\1', dict_output[k][i])
            temp += '</review_summary></review></reviews>'
            rs.append(temp)
    #put all results into the value_ordered in order
    for m in range(len(pi)):
        value_ordered += pi[m]
        value_ordered += rd[m]
        value_ordered += hn[m]   
        value_ordered += rt[m]
        value_ordered += rs[m]
    #put the new order of the value into the corresponding dictionary
    dict_output[k] = value_ordered

# 6. Compare the review date
1. Find the review date for all reviews
2. Cut the date to year, and compare year, then month, last compare the day

In [5]:
#extract all date in the value of the dictionary
for k,v in dict_output.items():
    latest_date = []
    latest_date = re.findall(r'<review_date>(\d{2} \d{1,2}, \d{4})', dict_output[k])
    #distinguish the date according to the length, if length is 1, then that is the latest review, or compare the year
    if len(latest_date) == 1:
        dict_output[k] = '<latest_review_date>' + latest_date[0] + '</latest_review_date>' + dict_output[k]
    else:
        #cut the year for the last four digits, and record the largest one
        latest_year = 0
        for i in latest_date:
            if int(i[-4:]) > latest_year:
                latest_year = int(i[-4:])
        #delete all elements with smaller than the largest year, if the length of the list is longer than 1, compare the month
        for i in range(len(latest_date))[::-1]:
            if int(latest_date[i][-4:]) < latest_year:
                latest_date.remove(latest_date[i])
        if len(latest_date) == 1:
            dict_output[k] = '<latest_review_date>' + latest_date[0] + '</latest_review_date>' + dict_output[k]
        else:
            #cut the month for the first two digits, and record the largest one
            latest_month = 0
            for i in latest_date:
                if int(i[:2]) > latest_month:
                    latest_month = int(i[:2])
            #delete all elements with smaller than the largest month, if the length of the list is longer than 1, compare the day
            for i in range(len(latest_date))[::-1]:
                if int(latest_date[i][:2]) < latest_month:
                    latest_date.remove(latest_date[i])
            if len(latest_date) == 1:
                dict_output[k] = '<latest_review_date>' + latest_date[0] + '</latest_review_date>' + dict_output[k]
            else:
                #copy the list of the day. first cut last six bits and then cut the first three bits to only keep the day, since the day is one or two digits
                latest_day = 0
                latest_date_cut = latest_date
                for i in range(len(latest_date_cut)):
                    latest_date_cut[i] = latest_date_cut[i].replace(latest_date_cut[i][-6:], '')
                    latest_date_cut[i] = latest_date_cut[i].replace(latest_date_cut[i][:3], '')
                for i in latest_date_cut:
                    if int(i) > latest_day:
                        latest_day = int(i)
                #delete all elements with smaller than the largest day, if the length of the list is longer than 1, change it to the set to delete the same date
                for i in range(len(latest_date))[::-1]:
                    if int(latest_date_cut[i]) < latest_day:
                        latest_date.remove(latest_date[i])
                latest_date = set(latest_date)
                latest_date = list(latest_date)
                dict_output[k] = '<latest_review_date>' + latest_date[0] + '</latest_review_date>' + dict_output[k]


# 7. Modify the format to xml
1. Change the key of the dictionary to xml format
2. Modify the beginning and end

In [6]:
#create a new dictionary to change the key
dict_key_changed = {}
xml_output = ''
#change the key to xml format and then add all keys and values to xml_output
for k,v in dict_output.items():
    new_key = ''
    new_key = k[14:]
    new_key = '</user><user name="' + str(new_key) + '">'
    dict_key_changed[new_key] = v
    xml_output += new_key
    xml_output += v

#Change the beginning and end to xml
xml_output = xml_output[7:]
xml_output = '<users>' + xml_output + '</user></users>'
xml_output = re.sub(r'</review_summary></review></reviews><reviews><review><productID>', r'</review_summary></review><review><productID>', xml_output)

# 8. Output to the file

In [7]:
text_file = open(r'D:/32049145.xml', 'w')

text_file.write(xml_output)
             
text_file.close()

# Reference List:
1. https://stackoverflow.com/questions/730133/what-are-invalid-characters-in-xml
2. https://stackoverflow.com/questions/13046240/parseerror-not-well-formed-invalid-token-using-celementtree